# FLORT

### Purpose
The purpose of this notebook is to calculate the gross range user min/max values for populating QARTOD parameter tables for data streams for OOI - CGSN data streams for the two-wavelength (FLORD) chlorophyll/fluorescence instrument. The FLORD instruments are deployed both on wire-following-profilers (coastal and global) as well as inductively-via-ctdbps attached to the wire on global surface moorings.

This requires the following steps:
1. Identify CGSN data streams in OOINet
2. Access the relevant data 
3. Process & combine the data from the different delivery methods and streams into a single dataset
4. Add annotations and remove data flagged as bad
5. Save the results

### Test Parameters and Values


| Class | Dataset Name | OOINet Name | Sensor Range |
| ----- | ------------ | ----------- | ------------ |
| FLORD | | fluorometric_chlorophyll_a | 0 - 30 ug/L |
|       | | total_volume_scattering_coefficient | 0 - 5 m$^{-1}$ |

#### Import libraries

In [ ]:
# Import libraries
import os, sys, datetime, pytz, re
import dateutil.parser as parser
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import gc
import json
warnings.filterwarnings("ignore")

In [ ]:
from dask.diagnostics import ProgressBar

#### Import the ```ooinet``` M2M toolbox

In [ ]:
# Import the OOINet M2M tool
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/")
from ooinet import M2M
from ooinet.utils import convert_time, ntp_seconds_to_datetime, unix_epoch_time
from ooinet.Instrument.common import process_file, add_annotation_qc_flag

#### Import ```ooi_data_explorations``` toolbox

In [ ]:
sys.path.append("/home/areed/Documents/OOI/oceanobservatories/ooi-data-explorations/python/")
from ooi_data_explorations.common import get_annotations, get_vocabulary, load_gc_thredds, add_annotation_qc_flags
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.uncabled.process_flord import flord_datalogger, flord_instrument
from ooi_data_explorations.qartod.qc_processing import identify_blocks, create_annotations, process_gross_range, \
    process_climatology, woa_standard_bins, inputs, ANNO_HEADER, CLM_HEADER, GR_HEADER#### Import plotting and visualization tools

#### Import plotting and visualization tools

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sys.path.append("../../qartod/")
from qartod.plotting import plot_climatology, plot_gross_range, plot_data_variable

---
## Identify Data Streams
This section is necessary to identify all of the data stream associated with a specific instrument. This can be done by querying UFrame and iteratively walking through all of the API endpoints. The results are saved into a csv file so this step doesn't have to be repeated each time.

First, set the instrument to search for using OOI terminology:

In [ ]:
instrument = "FLOR"

### Query OOINet for Data Streams <br>
If the data streams for a given instrument have not yet been identified from OOINet, then want to query OOINet for the data sets and save them to the local memory:

In [ ]:
try:
    datasets = pd.read_csv(f"../data/{instrument}_datasets.csv")
except:
    datasets = OOINet.search_datasets(instrument=instrument, English_names=True)
    datasets.to_csv(f"../data/{instrument}_datasets.csv", index=False)
datasets.head()

Separate out the CGSN datasets from the EA and RCA datasets:

In [ ]:
cgsn = datasets["array"].apply(lambda x: True if x.startswith(("CP","GA","GI","GP","GS")) else False)
datasets = datasets[cgsn]
flord = datasets["refdes"].apply(lambda x: True if "FLORD" in x else False)
datasets = datasets[flord]
datasets.head()

Drop all of the vehicle reference designators (those with "MOAS" in them)

In [ ]:
moas = datasets["array"].apply(lambda x: True if "MOAS" in x else False)
datasets = datasets[~moas]
datasets

---
## Single Reference Designator
The reference designator acts as a key for an instrument located at a specific location. First, select a reference designator (refdes) to request data from OOINet.

In [ ]:
reference_designators = sorted(datasets["refdes"])
print("Number of reference designators: " + str(len(reference_designators)))
for refdes in reference_designators:
    print(refdes)

In [ ]:
k=3
refdes = reference_designators[k]
print(refdes)

#### Sensor Vocab
The vocab provides information about the instrument model and type, its location (with descriptive names), depth, and manufacturer. Get the vocab for the given reference designator.

In [ ]:
vocab = M2M.get_vocab(refdes)
vocab

#### Sensor Deployments
Download the deployment information for the selected reference designator:

In [ ]:
deployments = M2M.get_deployments(refdes)
deployments

#### Sensor Data Streams
Next, select the specific data streams for the given reference designator

In [ ]:
datastreams = M2M.get_datastreams(refdes)
datastreams

---
## Metadata 
The metadata contains the following important key pieces of data for each reference designator: **method**, **stream**, **particleKey**, and **count**. The method and stream are necessary for identifying and loading the relevant dataset. The particleKey tells us which data variables in the dataset we should be calculating the QARTOD parameters for. The count lets us know which dataset (the recovered instrument, recovered host, or telemetered) contains the most data and likely has the best record to use to calculate the QARTOD tables. 

In [ ]:
metadata = M2M.get_metadata(refdes)
metadata

#### Sensor Parameters
Each instrument returns multiple parameters containing a variety of low-level instrument output and metadata. However, we are interested in science-relevant parameters for calculating the relevant QARTOD test limits. We can identify the science parameters based on the preload database, which designates the science parameters with a "data level" of L1 or L2. 

Consequently, we through several steps to identify the relevant parameters. First, we query the preload database with the relevant metadata for a reference designator. Then, we filter the metadata for the science-relevant data streams. 

In [ ]:
def filter_science_parameters(metadata):
    """This function returns the science parameters for each datastream"""
    
    def filter_parameter_ids(pdId, pid_dict):
        data_level = pid_dict.get(pdId)
        if data_level is not None:
            if data_level > 0:
                return True
            else:
                return False
        else:
            return False
    
    # Filter the parameters for processed science parameters
    data_levels = M2M.get_parameter_data_levels(metadata)
    mask = metadata["pdId"].apply(lambda x: filter_parameter_ids(x, data_levels))
    metadata = metadata[mask]

    return metadata

def filter_metadata(metadata):
    science_vars = filter_science_parameters(metadata)
    # Next, eliminate the optode temperature from the stream
    mask = science_vars["particleKey"].apply(lambda x: False if "temp" in x else True)
    science_vars = science_vars[mask]
    science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
    science_vars = science_vars.reset_index()
    science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
    science_vars = science_vars.explode(column="particleKey")
    return science_vars

In [ ]:
science_vars = filter_science_parameters(metadata)
science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
science_vars = science_vars.reset_index()
science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
science_vars

---
## Load the Data
Next, load the data from the different delivery methods into a single dataset.

In [ ]:
def trim_overlaps(ds, deployments):
    """Trim overlapping deployment data (necessary to use xr.open_mfdataset)"""
    # --------------------------------
    # Second, get the deployment times
    deployments = deployments.sort_values(by="deploymentNumber")
    deployments = deployments.set_index(keys="deploymentNumber")
    # Shift the start times by (-1) 
    deployEnd = deployments["deployStart"].shift(-1)
    # Find where the deployEnd times are earlier than the deployStart times
    mask = deployments["deployEnd"] > deployEnd
    # Wherever the deployEnd times occur after the shifted deployStart times, replace those deployEnd times
    deployments["deployEnd"][mask] = deployEnd[mask]
    deployments["deployEnd"] = deployments["deployEnd"].apply(lambda x: pd.to_datetime(x))
    
    # ---------------------------------
    # With the deployments info, can write a preprocess function to filter 
    # the data based on the deployment number
    depNum = np.unique(ds["deployment"])
    deployInfo = deployments.loc[depNum]
    deployStart = deployInfo["deployStart"].values[0]
    deployEnd = deployInfo["deployEnd"].values[0]
    
    # Select the dataset data which falls within the specified time range
    ds = ds.sel(time=slice(deployStart, deployEnd))
    
    return ds

In [ ]:
def preprocess_datalogger(ds):
    ds = process_file(ds)
    ds = trim_overlaps(ds, deployments)
    ds = flord_datalogger(ds)
    gc.collect()
    return ds

def preprocess_instrument(ds):
    ds = process_file(ds)
    ds = trim_overlaps(ds, deployments)
    ds = flord_instrument(ds)
    gc.collect()
    return ds

def preprocess_wfp(ds):
    ds = process_file(ds)
    ds = trim_overlaps(ds, deployments)
    ds = flord_wfp(ds)
    gc.collect()
    return ds

In [ ]:
# Filter out the "metadata" datastreams; use only the regular dataset
mask = datastreams["stream"].apply(lambda x: False if "metadata" in x or "blank" in x else True)
datastreams = datastreams[mask]
datastreams

---
## Download Data
To access data, there are two applicable methods. The first is to download the data and save the netCDF files locally. The second is to access and process the files remotely on the THREDDS server, without having to download the data.

In [ ]:
# Get the available datasets
for index in datastreams.index:
    # Get the method and stream
    method = datastreams.loc[index]["method"]
    stream = datastreams.loc[index]["stream"]

    # Get the URL - first try the goldCopy thredds server
    thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

    # Get the catalog
    catalog = M2M.get_thredds_catalog(thredds_url)

    # Clean the catalog
    catalog = M2M.clean_catalog(catalog, stream, deployments)
    
    # Get the links to the THREDDs server and load the data
    dodsC = M2M.URLS["goldCopy_dodsC"]
    
    # Not all datasets have made it into the goldCopy THREDDS - in that case, need to request
    # from OOINet
    if len(catalog) == 0:
        # Get the URL - first try the goldCopy thredds server
        thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=False)

        # Get the catalog
        catalog = M2M.get_thredds_catalog(thredds_url)

        # Clean the catalog
        catalog = M2M.clean_catalog(catalog, stream, deployments)

        # Get the links to the THREDDs server and load the data
        dodsC = M2M.URLS["dodsC"]
    
    # Now load the data
    if method == "telemetered":
        tele_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        if "WFP" in refdes:
            with ProgressBar():
                tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess_wfp, parallel=True)
        else:
            with ProgressBar():
                tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess_datalogger, parallel=True)
    elif method == "recovered_host":
        host_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            host_data = xr.open_mfdataset(host_files, preprocess=preprocess_datalogger, parallel=True)
    elif method == "recovered_inst":
        inst_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            inst_data = xr.open_mfdataset(inst_files, preprocess=preprocess_instrument, parallel=True)
    elif method == "recovered_wfp":
        wfp_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        wfp_files = [f for f in wfp_files if "blank" not in f]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            host_data = xr.open_mfdataset(wfp_files, preprocess=preprocess_wfp, parallel=True)
    else:
        pass

#### Combine the datasets into a single dataset

In [ ]:
data = combine_datasets(tele_data, host_data, None, None)
data

**Clean up workspace variables and free up memory**

In [ ]:
host_data.close()
tele_data.close()
del tele_data, host_data
gc.collect()

---
## Process the FLORD
The FLORD data needs extensive reprocessing before we can generate the qartod tables. So we download the data, reprocess it, and then will use the reprocessed data to calculate the statistics for the table.

In [ ]:
def add_hitl_qc_flags(data, refdes, annotations):
    site, node, sensor = refdes.split("-",2)
    # create boolean arrays of the data marked as "fail" by the quality checks and generate initial
    # HITL annotations that can be combined with system annotations to create a cleaned up data set
    # prior to calculating the QARTOD test values
    if node == 'WFP01':
        index = 10  # decimate the WFP data so we can process it
    else:
        index = 1
    if "estimated_chlorophyll_qc_summary_flag" in data.variables:
        chl_fail = data.estimated_chlorophyll_qc_summary_flag.where(data.estimated_chlorophyll_qc_summary_flag > 3).notnull()
        blocks = identify_blocks(chl_fail[::index], [18, 72])
        chl_hitl = create_annotations(site, node, sensor, blocks)
        chl_hitl['parameters'] = [[22, 1141] for i in chl_hitl['parameters']]
    else:
        chl_hitl = None

    if "fluorometric_cdom_qc_summary_flag" in data.variables:
        cdom_fail = data.fluorometric_cdom_qc_summary_flag.where(data.fluorometric_cdom_qc_summary_flag > 3).notnull()
        blocks = identify_blocks(cdom_fail[::index], [18, 72])
        cdom_hitl = create_annotations(site, node, sensor, blocks)
        cdom_hitl['parameters'] = [[23, 1143] for i in cdom_hitl['parameters']]
    else:
        cdom_hitl = None

    if "beta_700_qc_summary_flag" in data.variables:
        beta_fail = data.beta_700_qc_summary_flag.where(data.beta_700_qc_summary_flag > 3).notnull()
        blocks = identify_blocks(beta_fail[::index], [18, 72], 24)
        beta_hitl = create_annotations(site, node, sensor, blocks)
        beta_hitl['parameters'] = [[24, 25, 1139] for i in beta_hitl['parameters']]
    else:
        beta_hitl = None

    # combine the different dictionaries into a single HITL annotation dictionary for later use
    hitl = chl_hitl.copy()
    for d in (cdom_hitl, beta_hitl):
        if d is None:
            pass
        else:
            for key, value in d.items():
                hitl[key] = hitl[key] + d[key]

    # get the current system annotations for the sensor
    annotations = pd.DataFrame(annotations)
    if not annotations.empty:
        annotations = annotations.drop(columns=['@class'])
        annotations['beginDate'] = pd.to_datetime(annotations.beginDT, unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S')
        annotations['endDate'] = pd.to_datetime(annotations.endDT, unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S')

    # append the fail annotations to the existing annotations
    annotations = annotations.append(pd.DataFrame(hitl), ignore_index=True, sort=False)

    # create an annotation-based quality flag
    data = add_annotation_qc_flags(data, annotations)

    # clean-up the data, NaN-ing values that were marked as fail in the QC checks and/or identified as a block
    # of failed data, and then removing all records where the rollup annotation (every parameter fails) was
    # set to fail.
    if "estimated_chlorophyll" in data.variables and chl_fail is not None:
        data['estimated_chlorophyll'][chl_fail] = np.nan
        if 'fluorometric_chl_a_annotations_qc_results' in data.variables:
            m = data.fluorometric_chl_a_annotations_qc_results == 4
            data['estimated_chlorophyll'][m] = np.nan

    if "fluorometric_cdom" in data.variables and cdom_fail is not None:
        data['fluorometric_cdom'][cdom_fail] = np.nan
        if 'fluorometric_cdom_annotations_qc_results' in data.variables:
            m = data.fluorometric_cdom_annotations_qc_results == 4
            data['fluorometric_cdom'][m] = np.nan

    if "beta_700" in data.variables and beta_fail is not None:
        data['beta_700'][beta_fail] = np.nan
        if 'total_volume_scattering_coefficient_annotations_qc_results' in data.variables:
            m = data.total_volume_scattering_coefficient_annotations_qc_results == 4
            data['beta_700'][m] = np.nan
            if "bback" in data.variables:
                data['bback'][m] = np.nan

    if 'rollup_annotations_qc_results' in data.variables:
        data = data.where(data.rollup_annotations_qc_results != 4)
        
    return data    

#### Annotations
The annotations associated with a specific reference designator may contain relevant information on the performance or reliability of the data for a given dataset. The annotations are downloaded from OOINet as a json and processed into a pandas dataframe. Each annotation may apply to the entire dataset, to a specific stream, or to a specific variable. With the downloaed annotations, we can use the information contained in the ```qcFlag``` column to translate the annotations into QC flags, which can then be used to filter out bad data. 

In [ ]:
annotations = M2M.get_annotations(refdes)

**Add the HITL QC Flags**

In [ ]:
data = add_hitl_qc_flags(data, refdes, annotations)
data

---
## Gross Range
The Gross Range QARTOD test consists of two parameters: a fail range which indicates when the data is bad, and a suspect range which indicates when data is either questionable or interesting. The fail range values are set based upon the instrument/measurement and associated calibration. For example, the conductivity sensors are calibration for measurements between 0 (freshwater) and 9 (highly-saline waters). The suspect range values are calculated based on the mean of the available data $\pm$3$\sigma$.

In [ ]:
from ooi_data_explorations.qartod.gross_range import GrossRange
from ooi_data_explorations.qartod.plotting import *
from ooi_data_explorations.qartod.qc_processing import format_gross_range, format_climatology

#### Test Parameters & Sensor Ranges
Map out the data variables in the data set to the data stream inputs and the associated sensor ranges

In [ ]:
test_parameters = {
    "estimated_chlorophyll": {
        "inp": "fluorometric_chlorophyll_a",
        "sensor_range": [0, 30],
    },
    "beta_700": {
        "inp": "total_volume_scattering_coefficient",
        "sensor_range": [0, 5]
    }
}

**Calculate the Gross Range values**

In [ ]:
site, node, sensor = refdes.split("-", 2)
gross_range_table = pd.DataFrame()

for param in test_parameters:
    sensor_range = test_parameters.get(param).get("sensor_range")
    inp = test_parameters.get(param).get("inp") 
    
    if param in data.variables:
        print(f"##### Calculating gross range for {param} #####")
        # Check if there is enough data
        if len(data[param].dropna(dim="time")) < 100:
            user_range = sensor_range
            source = "Not enough data to calculate user range."
        else:
            gross_range = GrossRange(sensor_range[0], sensor_range[1])
            gross_range.fit(data, param, check_normality=True)
            user_range = [gross_range.suspect_min, gross_range.suspect_max]
            source = gross_range.source
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            qc_dict = format_gross_range(inp, sensor_range, user_range, site, node, sensor, stream, source)
            gross_range_table = gross_range_table.append(qc_dict, ignore_index=True)
            
        # ------------------ Plot the gross range ------------------
        if data[param].time.size > 100000:
            try:
                subset = sorted(np.random.choice(data.time, 100000, replace=False))
                subset_data = data.sel(time=subset)
                plot_gross_range(subset_data, param, gross_range)
                del subset, subset_data
                gc.collect()
            except:
                pass
        else:
            try:
                plot_gross_range(data, param, gross_range**Calculate the Gross Range values**)
            except:
                pass

**Add the stream name and the source comment**

In [ ]:
gross_range_table['notes'] = ('User range based on data collected through {}.'.format("2021-01-01"))
gross_range_table

**Check the results**

In [ ]:
for ind in gross_range_table.index:
    print(gross_range_table.loc[ind]["qcConfig"])

**Save the gross range table**

In [ ]:
# Save the results
gross_range_table.to_csv(f"../results/gross_range/{refdes}.csv", index=False, columns=GR_HEADER)

---
## Climatology
For the climatology QARTOD test, First, we bin the data by month and take the mean. The binned-montly means are then fit with a 2 cycle harmonic via Ordinary-Least-Squares (OLS) regression. Ranges are calculated based on the 3$\sigma$ calculated from the OLS-fitting.  

In [ ]:
from ooi_data_explorations.qartod.climatology import Climatology

In [ ]:
def make_climatology_table(ds, param, tinp, zinp, sensor_range, depth_bins):
    """Function which calculates the climatology table based on the """
    
    climatologyTable = pd.DataFrame()
    
    if depth_bins is None:
        # Filter out the data outside the sensor range
        m = (ds[param] > sensor_range[0]) & (ds[param] < sensor_range[1]) & (~np.isnan(ds[param]))
        param_data = ds[param][m]
        
        # Fit the climatology for the selected data
        pmin, pmax = [0, 0]
        
        try:
            climatology = Climatology()
            climatology.fit(param_data)

            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Calculate the climatology data
            vmin = climatology.monthly_fit - climatology.monthly_std*3
            vmin = np.floor(vmin*10000)/10000
            for vind in vmin.index:
                if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                    vmin[vind] = sensor_range[0]
            vmax = climatology.monthly_fit + climatology.monthly_std*3
            for vind in vmax.index:
                if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                    vmax[vind] = sensor_range[1]
            vmax = np.floor(vmax*10000)/10000
            vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
            vspan = vdata.values.reshape(1,-1)

            # Build the climatology dataframe
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

        except:
            # Here is where to create nans if insufficient data to fit
            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Create a series filled with nans
            vals = []
            for i in np.arange(len(tspan)):
                vals.append([np.nan, np.nan])
            vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

            # Add to the data
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))
            
        del ds, vspan, tspan, zspan
        gc.collect()
        
    else:        
    # Iterate through the depth bins to calculate the climatology for each depth bin
        for dbin in depth_bins:
            # Get the pressure range to bin from
            pmin, pmax = dbin[0], dbin[1]

            # Select the data from the pressure range
            bin_data = data.where((data[zinp] >= pmin) & (data[zinp] <= pmax), drop=True)

            # sort based on time and make sure we have a monotonic dataset
            bin_data = bin_data.sortby('time')
            _, index = np.unique(bin_data['time'], return_index=True)
            bin_data = bin_data.isel(time=index)

            # Filter out the data outside the sensor range
            m = (bin_data[param] > sensor_range[0]) & (bin_data[param] < sensor_range[1]) & (~np.isnan(bin_data[param]))
            param_data = bin_data[param][m]

            # Fit the climatology for the selected data
            try:
                climatology = Climatology()
                climatology.fit(param_data)

                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Calculate the climatology data
                vmin = climatology.monthly_fit - climatology.monthly_std*3
                vmin = np.floor(vmin*10000)/10000
                for vind in vmin.index:
                    if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                        vmin[vind] = sensor_range[0]
                vmax = climatology.monthly_fit + climatology.monthly_std*3
                vmax = np.floor(vmax*10000)/10000
                for vind in vmax.index:
                    if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                        vmax[vind] = sensor_range[1]
                vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
                vspan = vdata.values.reshape(1,-1)

                # Build the climatology dataframe
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            except:
                # Here is where to create nans if insufficient data to fit
                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Create a series filled with nans
                vals = []
                for i in np.arange(len(tspan)):
                    vals.append([np.nan, np.nan])
                vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

                # Add to the data
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            del climatology, bin_data, vspan, tspan, zspan
            gc.collect()
    
    return climatologyTable, climatology

**Get the depth bins and filter based on max depth**

In [ ]:
if "WFP" in refdes:
    depth_bins = woa_standard_bins()
    pmax = data["depth"].max().values
    pmin = data["depth"].min().values
    mask = (depth_bins[:, 0] < pmax) | ((depth_bins[:, 0] < pmax) & (depth_bins[:, 1] > pmax)) | (depth_bins[:, 1] < pmin)
    depth_bins = depth_bins[mask]
else:
    depth_bins = None
    
depth_bins

**Calculate the climatology tables and associated lookup table**

In [ ]:
# Initialize the climatology lookup table
climatologyLookup = pd.DataFrame()

# Setup the Table Header
TBL_HEADER = ["[1,1]","[2,2]","[3,3]","[4,4]","[5,5]","[6,6]","[7,7]","[8,8]","[9,9]","[10,10]","[11,11]","[12,12]"]

# Set the subsite-node-sensor
subsite, node, sensor = refdes.split("-", 2)

# Iterate through the parameters
for param in parameters:
    if param in data.variables:
        # ----------------- Depth tables ---------------------
        # Get the sensor range of the parameter to test
        print(f"##### Calculating climatology for {param} #####")
        sensor_range = parameters.get(param)
        
        # Generate the climatology table with the depth bins
        climatologyTable, climatology = make_climatology_table(data, param, "time", "depth", sensor_range, depth_bins)
        
        # Get the variance and generate the source
        if "WFP" in refdes:
            source = "Climatology values are calculated from and applicable to standard depth bins."
        else:
            try:
                variance = float(np.round(climatology.regression['variance_explained']*100, 1))
            except:
                variance = 0.0
            source = f"The variance explained by the climatology mode is {variance}%"

        # Create the tableName
        tableName = f"{refdes}-{param}.csv"
        
        # Save the results
        climatologyTable.to_csv(f"../results/climatology/climatology_tables/{tableName}", header=TBL_HEADER)
        
        # ------------------ Lookup tables ------------------
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            qc_dict = {
                "subsite": subsite,
                "node": node,
                "sensor": sensor,
                "stream": stream,
                "parameters": {
                    "inp": inp,
                    "tinp": "time",
                    "zinp": "depth",
                },
                "climatologyTable": f"climatology_tables/{refdes}-{param}.csv",
                "source": source,
                "notes": "Climatology based on available data through 2021-01-01."
            }
            # Append to the lookup table
            climatologyLookup = climatologyLookup.append(qc_dict, ignore_index=True)
            
        # ------------------ Plot the climatology ------------------
        # Don't attempt to plot the WFP data
        if "WFP" in refdes:
            continue
        else:
            if data[param].time.size > 100000:
                try:
                    subset = sorted(np.random.choice(data.time, 100000, replace=False))
                    subset_data = data.sel(time=subset)
                    plot_climatology(subset_data, param, climatology)
                    del subset, subset_data
                    gc.collect()
                except:
                    pass
            else:
                try:
                    plot_climatology(data, param, climatology)
                except:
                    pass

**Check the last climatologyTable for reasonableness**

In [ ]:
climatologyTable

**Check the climatologyLookup table that all the entries made it in**

In [ ]:
climatologyLookup

**Save the climatologyLookup table**

In [ ]:
climatologyLookup.to_csv(f"../results/climatology/{refdes}.csv", index=False, columns=CLM_HEADER)